In [ ]:
import json
import os
import re
import math
import xml.etree.ElementTree as et
import numpy as np
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
path="yourdirect/VOC3000/Annotations/3000_"
num=3147
def get_namespace(element):  # 获取节点的namespace
    m = re.match('\{.*\}', element.tag)
    return m.group(0) if m else ''
data_set=[]
monum=0
mrnum=0
map_obj={}
map_rel={}
for i in range(num):
    #if i % 10 ==0:
    #    print(i)
    strnum=str(i).zfill(6)
    tpath=path+strnum+".xml"
    if not os.path.exists(tpath):
        continue
    tree=et.parse(tpath)
    
    root=tree.getroot()
    ns=get_namespace(root)
    #print(ns)
    #print(root.tag)
    img_data=[]
    obj=[]
    relation_dict=[]
    img_size=[]
    for child in root:
        #print(child.tag)
        
        if child.tag=="object":
            tmp=[]
            for cd in child:
                if cd.tag=="name":
                    #print(cd.text)
                    if cd.text not in map_obj.keys():
                        if cd.text.find("_")==-1:
                            map_obj[cd.text]=monum
                            monum+=1
                    tmp.append(cd.text)
                if cd.tag=="object_id":
                    tmp.append(cd.text)
                if cd.tag=="bndbox":
                    for n in cd:
                        tmp.append(n.text)
            #print(tmp)
            obj.append(tmp)
            continue
        if child.tag=="size":
            for cd in child:
                img_size.append(cd.text)
    obj_data=[]
    if len(obj)==3:
        flag=1
        relnum=0
        for h in range(len(obj)):
            if obj[h][0].find("_")!=-1:
                relnum+=1
                flag=0
                tmp=obj[h][0].split("_")
                a1=0
                a2=len(tmp)-1
                #这一步是因为数据集奇葩，15th person lying in beach object里面没有beach，吐了
                if tmp[a1] in map_obj.keys():
                    relation_dict.append(tmp[a1])
                else:
                    flag=1
                rela=tmp[a1+1]
                for o in range(a1+2,a2):
                    rela=rela+"_"+tmp[o]
                if rela not in map_rel.keys():
                    map_rel[rela]=mrnum
                    mrnum+=1
                relation_dict.append(rela)
                if tmp[a2] in map_obj.keys():
                    relation_dict.append(tmp[a2])
                else:
                    flag=1
            else:
                obj_data.append(obj[h])
        if flag==1 or relnum>1:
            continue
        img_data.append(obj_data)
        img_data.append(relation_dict)
        img_data.append(img_size)
        img_data.append(i)
        data_set.append(img_data)
print(map_obj)
print(map_rel)
print(data_set)


In [ ]:
def location_vec(subnum,objnum,relnum,img_size,sub_loc,obj_loc):
    
    sub_x_len=sub_loc[2]-sub_loc[0]
    sub_y_len=sub_loc[3]-sub_loc[1]
    obj_x_len=obj_loc[2]-obj_loc[0]
    obj_y_len=obj_loc[3]-obj_loc[1]
    img_len=math.sqrt(pow(img_size[0],2)+pow(img_size[1],2))
    
    #方位
    sub_ctrx=(sub_loc[0]+sub_loc[2])/2
    sub_ctry=(sub_loc[1]+sub_loc[3])/2
    obj_ctrx=(obj_loc[0]+obj_loc[2])/2
    obj_ctry=(obj_loc[1]+obj_loc[3])/2
    sub_x_refer=sub_x_len*0.33
    sub_y_refer=sub_y_len*0.33
    if sub_ctrx-obj_ctrx<-sub_x_refer:
        x_direct=-1
    elif sub_ctrx-obj_ctrx>sub_x_refer:
        x_direct=1
    else:
        x_direct=0
    if sub_ctry-obj_ctry<-sub_y_refer:
        y_direct=-1
    elif sub_ctry-obj_ctry>sub_y_refer:
        y_direct=1
    else:
        y_direct=0
    #距离
    dist_len=math.sqrt(pow((sub_ctrx-obj_ctrx),2)+pow((sub_ctry-obj_ctry),2))
    dist=0
    if dist_len>0.3*img_len:
        dist=3
    elif dist_len>0.2*img_len:
        dist=2
    elif dist_len>0.1*img_len:
        dist=1
    #重叠
    sub_x_len=sub_loc[2]-sub_loc[0]
    sub_y_len=sub_loc[3]-sub_loc[1]
    obj_x_len=obj_loc[2]-obj_loc[0]
    obj_y_len=obj_loc[3]-obj_loc[1]
    overlapx=0
    overlapy=0
    if sub_loc[0]-obj_loc[0]<0 and sub_loc[2]-obj_loc[2]>0:
        overlapx=-4
    elif (sub_loc[0]-obj_loc[0]<0 and sub_loc[2]-obj_loc[2]>-obj_x_len*0.1) or (sub_loc[0]-obj_loc[0]<obj_x_len*0.1 and sub_loc[2]-obj_loc[2]>0):
        overlapx=-3
    elif sub_loc[0]-obj_loc[0]<obj_x_len*0.1 and sub_loc[2]-obj_loc[2]>-obj_x_len*0.1:
        overlapx=-2
    elif sub_loc[0]-obj_loc[0]<obj_x_len*0.2 and sub_loc[2]-obj_loc[2]>-obj_x_len*0.2:
        overlapx=-1
    if obj_loc[0]-sub_loc[0]<0 and obj_loc[2]-sub_loc[2]>0:
        overlapx=4
    elif ((obj_loc[0]-sub_loc[0]<0 and obj_loc[2]-sub_loc[2]>-sub_x_len*0.1) or (obj_loc[0]-sub_loc[0]<sub_x_len*0.1 and obj_loc[2]-sub_loc[2]>0)):
        overlapx=3
    elif obj_loc[0]-sub_loc[0]<sub_x_len*0.1 and obj_loc[2]-sub_loc[2]>-sub_x_len*0.1:
        overlapx=2
    elif obj_loc[0]-sub_loc[0]<sub_x_len*0.2 and obj_loc[2]-sub_loc[2]>-sub_x_len*0.2:
        overlapx=1

    if sub_loc[1]-obj_loc[1]<0 and sub_loc[3]-obj_loc[3]>0:
        overlapy=-4
    elif ((sub_loc[1]-obj_loc[1]<0 and sub_loc[3]-obj_loc[3]>-obj_y_len*0.1) or (sub_loc[1]-obj_loc[1]<obj_y_len*0.1 and sub_loc[3]-obj_loc[3]>0)):
        overlapy=-3
    elif sub_loc[1]-obj_loc[1]<obj_y_len*0.1 and sub_loc[3]-obj_loc[3]>-obj_y_len*0.1:
        overlapy=-2
    elif sub_loc[1]-obj_loc[1]<obj_y_len*0.2 and sub_loc[3]-obj_loc[3]>-obj_y_len*0.2:
        overlapy=-1
    if obj_loc[1]-sub_loc[1]<0 and obj_loc[3]-sub_loc[3]>0:
        overlapy=4
    elif (obj_loc[1]-sub_loc[1]<0 and obj_loc[3]-sub_loc[3]>-sub_y_len*0.1) or (obj_loc[1]-sub_loc[1]<sub_y_len*0.1 and obj_loc[3]-sub_loc[3]>0):
        overlapy=3
    elif obj_loc[1]-sub_loc[1]<sub_y_len*0.1 and obj_loc[3]-sub_loc[3]>-sub_y_len*0.1:
        overlapy=2
    elif obj_loc[1]-sub_loc[1]<sub_y_len*0.2 and obj_loc[3]-sub_loc[3]>-sub_y_len*0.2:
        overlapy=1
    #len=8
    return [subnum,objnum,x_direct,y_direct,overlapx,overlapy,dist]

train_x=[]
train_y=[]
print("len=",len(data_set))
for i in range(len(data_set)):
    print(i)
    Subject=data_set[i][1][0]
    Object=data_set[i][1][2]
    Relation=data_set[i][1][1]
    imgnum=data_set[i][3]
    print(Subject,Relation,Object,imgnum)
    subnum=map_obj[Subject]
    objnum=map_obj[Object]
    relnum=map_rel[Relation]
    img_size=data_set[i][2]
    idx=0
    if data_set[i][0][0][0]==Subject:
        idx=0
    else:
        idx=1
    sub_loc=data_set[i][0][idx][1:]
    obj_loc=data_set[i][0][1-idx][1:]
    for j in range(len(img_size)):
        img_size[j]=float(img_size[j])
    for j in range(len(sub_loc)):
        sub_loc[j]=float(sub_loc[j])
    for j in range(len(obj_loc)):
        obj_loc[j]=float(obj_loc[j])
    vec=location_vec(subnum,objnum,relnum,img_size,sub_loc,obj_loc)
    train_x.append(vec)
    train_y.append(relnum)

In [ ]:
param_grid ={'C':[1e3,5e3,1e4,5e4,1e5],#C是对错误的惩罚
              'gamma':[0.0001,0.0005,0.001,0.005,0.01,0.1],}#gamma核函数里多少个特征点会被使用}#对参数尝试不同的值
clf = GridSearchCV(SVC(kernel='rbf'),param_grid)
x_train,x_test,y_train,y_test=train_test_split(train_x,train_y,test_size=0.2)

clf=clf.fit(x_train,y_train)
y=clf.predict(x_test)
cnt=0
for i in range(len(y)):
    print(y[i]," ",y_test[i])
    if y[i]==y_test[i]:
        cnt+=1
print(cnt/len(y))